## Deep Research

One of the classic cross-business Agentic use cases!
A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities.

In [10]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [11]:
load_dotenv(override=True)

True

In [12]:
# Configuration & credentials
llm_model = "gpt-4o-mini"
sender_email = os.environ["SENDER_EMAIL"]
receiver_email = os.environ["RECEIVER_EMAIL"]

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

In [13]:
research_instructions = """You are a research assistant. Given a search term,
you search the web for that term and produce a concise summary of the results.
The summary must be less than 100 words. Capture the main points.
This will be consumed by someone synthesizing a report, so it's vital you capture
the essence and ignore any fluff. Do not include any additional commentary other
than the summary itself."""

search_agent = Agent(name="Search agent",
                     instructions=research_instructions,
                     tools=[WebSearchTool(search_context_size="low")],
                     model=llm_model,
                     model_settings=ModelSettings(tool_choice="required"))

### We will now use Structured Outputs, and include a description of the fields

In [14]:
HOW_MANY_SEARCHES = 3

planner_instructions = (f"You are a helpful research assistant. Given a query, "
                        f"come up with a set of web searches to respond with the best answer of the user query. Output {HOW_MANY_SEARCHES} terms to query for.")

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(name="PlannerAgent",
                      instructions=planner_instructions,
                      model=llm_model,
                      output_type=WebSearchPlan)

In [15]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender_email) # Change this to your verified email
    to_email = To(receiver_email) # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [16]:
email_instructions = """You are able to send a nicely formatted HTML email based on a detailed report. You will be provided with a detailed report. You should use your tool to send one email, providing the report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(name="Email agent",
                    instructions=email_instructions,
                    tools=[send_email],
                    model=llm_model)

In [17]:
writer_instructions = """You are a senior researcher tasked with writing a cohesive report for a research query. You will be provided with the original query, and some initial research done by a research assistant.

You should first come up with an outline for the report that describes the structure and
flow of the report. Then, generate the report and return that as your final output. The
final output should be in markdown format, and it should be lengthy and detailed. Aim for
1-2 pages of content, at least 200 words."""


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=writer_instructions,
    model=llm_model,
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [18]:
async def plan_searches(user_query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {user_query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(plan_search: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in plan_search.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    search_input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, search_input)
    return result.final_output

### The next 2 functions write a report and email it

In [19]:
async def write_report(user_query: str, search_result: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    writer_input = f"Original query: {user_query}\nSummarized search results: {search_result}"
    result = await Runner.run(writer_agent, writer_input)
    print("Finished writing report")
    return result.final_output

async def send_email(writer_report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, writer_report.markdown_report)
    print("Email sent")
    return writer_report

### Showtime!

In [20]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


### As always, take a look at the trace

https://platform.openai.com/traces